<a href="https://colab.research.google.com/github/Howl06/practice/blob/main/linebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir -p /content/drive/MyDrive/linebot

In [3]:
cd /content/drive/MyDrive/linebot

/content/drive/MyDrive/linebot


In [4]:
!pip install flask-ngrok
!pip install flask
!pip install pyngrok
!pip install line-bot-sdk
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.2/752.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [7]:
import configparser

def create_default_config():
    """Creates a default configuration file."""
    config = configparser.ConfigParser()
    config["API"] = {
        'ngrok_token': '',
        'openai_api_key': ''
    }
    config["LINE"] = {
        'line_channel_access_token': '',
        'line_channel_secret': ''
    }
    config["PROMPT"] = {'system_prompt': ''}

    with open('config.ini', 'w') as configfile:
        config.write(configfile)

In [9]:
import configparser

config_dict = {"ngrok_token": "",
        "openai_api_key": "",
        "Line_Channel_Access_Token": "",
        "Line_Channel_Secret": "",
        'system_prompt': "",
        }
try:
    config = configparser.ConfigParser()
    config.read("config.ini")
    ngrok_token = config["API"]["ngrok_Token"]
    openai_api_key = config["API"]["openai_api_key"]
    Line_Channel_Access_Token = config["LINE"]["Line_Channel_Access_Token"]
    Line_Channel_Secret = config["LINE"]["Line_Channel_Secret"]
    system_prompt = config["PROMPT"]["system_prompt"]
    print("config.ini OK!")
except Exception as e:
    print(e)
    create_default_config
    print(" config.ini is missing")

config.ini OK!


In [10]:
from pyngrok import ngrok, conf
import getpass
conf.get_default().auth_token = ngrok_token #getpass.getpass()
# enter your ngrok Token

In [13]:
import time
ngrok_url=''
#自動更新WebhookURL
def auto_update_webhook_url() -> None:
    while(1):
        time.sleep(5) #等候5秒讓ngrok完成註冊新網址
        try:
            import json
            import requests
            #取得ngrok最新產生的url
            self_url = "http://localhost:4040/api/tunnels"
            res = requests.get(self_url)
            res_unicode = res.content.decode("utf-8")
            res_json = json.loads(res_unicode)
            ngrok_url = res_json["tunnels"][0]["public_url"]

            #開始更新
            line_put_endpoint_url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
            data = {"endpoint": ngrok_url + '/callback'}
            headers = {
                "Authorization": "Bearer " + Line_Channel_Access_Token ,
                "Content-Type": "application/json"
                    }
            print(data)
            res = requests.put(line_put_endpoint_url, headers=headers, json=data)
        # 檢查回應狀態碼
            if res.status_code == 200:
                print("WebhookURL更新成功！")
                break
                #強制關閉threading
                x._stop()
            else:
                print("WebhookURL更新失敗")
        except:
            pass
            #強制關閉threading
            break
            x._stop()


In [14]:
from openai import OpenAI




def chatgpt_respose(user_message:str, openai_api_key:str, system_prompt:str) -> str:
    try:
        client = OpenAI(api_key=openai_api_key)
        completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
            ])
        print(completion.choices[0].message)
        return completion.choices[0].message.content
    except Exception as e:
        print(e)
        return "抱歉目前不提供服務"



In [ ]:
from flask import *
from flask_ngrok import run_with_ngrok
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import ( InvalidSignatureError)
from linebot.models import *
import time
import threading

#Messaging API settings/Channel access token
line_bot_api = LineBotApi(Line_Channel_Access_Token)

#Basic settings/Channel secret
handler = WebhookHandler(Line_Channel_Secret)


app = Flask(__name__)
# run_with_ngrok(app)

public_url = ngrok.connect(5000).public_url
app.config["BASE_URL"] = public_url

@app.route("/")
def home():
    return "首頁在這"

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

#實際處理訊息的部份
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    replyMsg=[TextSendMessage(text=chatgpt_respose(event.message.text, openai_api_key, system_prompt))]#回覆文字=收到的文字
    line_bot_api.reply_message(
        event.reply_token,
        replyMsg
    )


if __name__ == "__main__":
    #啟動執行緒以更新webhook_url網址
    #   threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()
    x = threading.Thread(target=auto_update_webhook_url)
    x.start()
    app.run()